In [1]:
import os 
#data_location = input("Enter Path Location of data")
os.chdir("/home/mitchell/Documents/masters/masters/scripts/")
import galaxy_pairs
import cmb
os.chdir("/home/mitchell/Documents/masters/masters/data")
cwd = os.getcwd()
print(cwd)

pywtl packages could not be imported.  This may break plotting code.
FlatLambdaCDM(name="Planck15", H0=67.7 km / (Mpc s), Om0=0.307, Tcmb0=2.725 K, Neff=3.05, m_nu=[0.   0.   0.06] eV, Ob0=0.0486)
/home/mitchell/Documents/masters/masters/data


In [2]:
import numpy as np
import scipy as sp
import astropy as ap
import glob
import pandas as pd
import matplotlib.pyplot as plt
from scipy.integrate import quad
import scipy.constants as const
from astropy import constants as ap_const
import scipy.ndimage 
import PIL
import sptpol_software as sps
from astropy.io import fits
from astropy.cosmology import Planck15
from astropy import units as u
import scipy.spatial.distance as dist
print(Planck15)
cosmo= Planck15

FlatLambdaCDM(name="Planck15", H0=67.7 km / (Mpc s), Om0=0.307, Tcmb0=2.725 K, Neff=3.05, m_nu=[0.   0.   0.06] eV, Ob0=0.0486)


In [3]:
from sptpol_software.util.tools import stat
from sptpol_software.observation import *
import sptpol_software.observation as obs
import sptpol_software as sps
import sptpol_software.observation.sky
from sptpol_software.util import files

In [4]:
T_cmb = 2.725
freqs_ghz = [93.2000,147.700]
beam_norm_correction = [1./0.99673, 1./0.99470, 1./1. ] 
cal_factors = [0.9097,0.7765] # no 220 is why last is zero. 
#pol_cal_factors = pol_cal_factors_800 * beam_norm_correction

In [ ]:
glob.glob('*.h5')

In [ ]:
map_150ghz = files.read("ra0dec-57p5_sum5000_150ghz.h5")
map_90ghz = files.read("ra0dec-57p5_sum5000_090ghz.h5")

In [ ]:
# map_150ghz = pol_cal_factors_800[0]*map_150ghz
# map_90ghz = pol_cal_factors_800[1]*map_90ghz

In [ ]:
# map_150ghz.getSubmap([5,5], center_offset=[0,0], units='degree').drawImage(bw=False,vmax=0)
# map_90ghz.getSubmap([10,10], center_offset=[0,0], units='degree').drawImage(bw=False)

In [ ]:
# diff_T = map_150ghz-map_90ghz
# diff_T.getSubmap([5,5], center_offset=[1,1], units='degree').drawImage(bw=False)

In [ ]:
# def convert_ghz_to_y(freq_power):
#     x = freq_power/56.85  # x = h v / k_B T_CMB
#     return((x/np.tanh(x/2)) - 4)
print(cmb.convert_ghz_to_y(90))
print(cmb.convert_ghz_to_y(150))

In [ ]:
freq_range = np.arange(20,1000,1)
x = freq_range/56.85
y_param = [(ex/np.tanh(ex/2) - 4) for ex in x]
plt.semilogx(freq_range,y_param,label='Y')
I_v = [(ex**4 * np.exp(ex))/(np.exp(ex)-1)**2 * (ex/np.tanh(ex/2) - 4) for ex in x]
plt.semilogx(freq_range,I_v,label='Intensity')
plt.axhline(0, color='black')
plt.title('Frequency Dependance of SZ Effect')
plt.legend()

In [ ]:
# map_150ghz_array = map_150ghz.getTOnly().map
# print(type(map_150ghz))

In [ ]:
# freq_scaling = convert_ghz_to_y(150)-convert_ghz_to_y(90)
# print(freq_scaling)
# freq_scaling_fac = 1.0/freq_scaling
# print(freq_scaling_fac)

In [ ]:
#y_map = sptpol_software.observation.sky.Map
# y_map = freq_scaling_fac*(map_150ghz-map_90ghz)
#/(convert_ghz_to_y(150)-convert_ghz_to_y(90))

In [ ]:
# y_map.getSubmap([5,5], center_offset=[0,0], units='degree').drawImage(bw=True)
# print("Functionally a factor of 2 conversion, look at scale")
# diff_T.getSubmap([2,2], center_offset=[0,0], units='degree').drawImage(bw=False)
y_map_array = cmb.get_y_map([map_150ghz,map_90ghz],cal_factors,freqs_ghz)


In [ ]:
# y_map.drawImage(bw=False,vmax=-0.00001,vmin=0.00001)

In [ ]:
# np.shape(y_map)

In [ ]:
# y_map.writeToHDF5('y_map.h5',overwrite=True,use_compression=False)
# y_map_array = y_map.getTOnly().map
print(y_map_array)
# np.save('y_map', y_map_array)

In [ ]:
plt.imshow(y_map_array,vmax=0.00001,vmin=-0.00001)

In [5]:
# red_mapper_cat = fits.open("DES_Y1A1_3x2pt_redMaGiC_zerr_CATALOG.fits")
from astropy.table import Table
dat = Table.read('DES_Y1A1_3x2pt_redMaGiC_zerr_CATALOG.fits', format='fits')
df = dat.to_pandas()

In [6]:
# red_mapper_cat.info()
df

,ID,RA,DEC,ZREDMAGIC,ZREDMAGIC_E,weight
0,3050237433,76.650392,-56.996599,0.150009,0.009600,1.051875
1,3119536009,10.657494,-44.892612,0.150010,0.009513,0.911657
2,3043842585,81.917004,-56.612052,0.150013,0.008653,0.937038
3,3034628499,89.812164,-52.779491,0.150034,0.008681,1.076568
4,3059228029,71.135057,-51.074555,0.150044,0.008685,1.048341
5,3119296447,11.913672,-42.342889,0.150051,0.008591,0.963716
6,3081078666,52.842355,-57.074867,0.150058,0.008494,0.917235
7,3075699929,57.693299,-48.650024,0.150062,0.008151,0.892103
8,3090139698,43.089595,-43.859265,0.150065,0.009137,0.942256
9,3057872589,70.976981,-44.950415,0.150070,0.008823,0.953896


In [7]:
# com_dists = cosmo.comoving_distance(df['ZREDMAGIC']).to_value(u.Mpc)
df['COMOVING'] = pd.Series(cosmo.comoving_distance(df['ZREDMAGIC']).to_value(u.Mpc))
df['COMOVING_E'] = pd.Series(cosmo.comoving_distance(df['ZREDMAGIC_E']).to_value(u.Mpc))

In [ ]:
help(galaxy_pairs.getPairs)

In [8]:
# print(df)
pairs = galaxy_pairs.getPairs(df,20)
# First index is first item in pair
# Second index in second item in pair
# Third Index is distance apart from each other


In [9]:
pairs_df = pd.DataFrame(pairs.T,columns = ['First_Loc','Second_Loc','Sep'])

In [10]:
# print(pairs)
pairs_df['First_Loc'] = pairs_df.First_Loc.astype(int)
pairs_df['Second_Loc'] = pairs_df.Second_Loc.astype(int)

In [11]:
pairs_df

,First_Loc,Second_Loc,Sep
0,645602,645787,19.440705
1,651449,652553,12.694922
2,645787,647190,17.268397
3,649428,649606,13.064617
4,649348,649809,19.667827
5,649876,650136,12.081126
6,649876,651213,14.828850
7,649405,649876,8.673922
8,649405,650136,9.708314
9,649809,650136,17.977599


In [38]:
import random
index = random.randint(1,len(df))
pos1 = [df['RA'][pairs_df['First_Loc'][index]],df['DEC'][pairs_df['First_Loc'][index]]]
# print(index)
# print(pos1)
# # pos2 = [df['RA'][pairs_df['Second_Loc'][0]],df['DEC'][pairs_df['Second_Loc'][0]]]
# # print(pos2)

In [ ]:
# df.loc[index]

In [39]:
# df.loc[pairs_df['Second_Loc'][0]]
def SubtractRaDec(radec_1, radec_2):
#     assert((len(radec_1) == 2)) #,"First enetered RaDec does not have two components")
#     assert((len(radec_2) == 2)) #,"Second enetered RaDec does not have two components")

    ra_1 = radec_1[0]
    dec_1 = radec_1[1]
    ra_2 = radec_2[0]
    dec_2 = radec_2[1]
    
    ra_diff = 180 - abs(abs(ra_1 - ra_2) - 180)
    
    dec_diff = 180 - abs(abs(dec_1 - dec_2) - 180)
    
    return([ra_diff,dec_diff])

In [40]:
# rel_pos_1 = [0-pos1[0],-57.5-pos1[1]]
rel_pos_1 = SubtractRaDec([0,-57.5],pos1)
print(rel_pos_1)
# print(rel_pos_1[0])
# [15.089213,5.669675000000012]

[38.25365499999998, 5.502987000000019]


In [ ]:
# y_map.getSubmap([1,1], center_offset=[rel_pos_1[0],rel_pos_1[1]], units='degree').drawImage(bw=False)

In [ ]:
print(np.max(df['RA']))
print(np.min(df['RA']))
print(np.max(df['DEC']))
print(np.min(df['DEC']))


In [ ]:
# np.savetxt('pairs.csv',pairs.T,delimiter=',')

In [ ]:
vec1 = galaxy_pairs.get_vec_distances(df['RA'][int(pairs[0][0])],df['DEC'][int(pairs[0][0])],df['COMOVING'][int(pairs[0][0])])
vec2 = galaxy_pairs.get_vec_distances(df['RA'][int(pairs[1][0])],df['DEC'][int(pairs[1][0])],df['COMOVING'][int(pairs[1][0])])
print(galaxy_pairs.euclideanDistance(vec1,vec2,3))

In [ ]:
# df.to_csv('DES_REDMAGIC_Manipulatedo.csv')

In [ ]:
# vec_unit_test = RaDec2XYZ(df['RA'],df['DEC'])
# print(vec_unit_test)
# print(np.shape(vec_unit_test))
# vec_unit_test2 = vec_unit_test[0]
# print(vec_unit_test2)

In [ ]:
help(sptpol_software.observation.sky.ang2Pix)

In [41]:
sptpol_software.observation.sky.ang2Pix(rel_pos_1,[0,-57.5],reso_arcmin=0.25,map_pixel_shape=[1320, 2520])

([array([-14461]), array([10398])], (array([False]), array([False])))

In [ ]:
# def get_subarray(array,centre, sqr_radius):
#     ''' 
#     Gets Sub Array with with an input centre and half width from an input Array
#     '''
#     x_cen = centre[0]
#     y_cen = centre[1]
#     sl_x = slice(x_cen-sqr_radius,x_cen+sqr_radius)
#     sl_y = slice(y_cen-sqr_radius,y_cen+sqr_radius)
    
#     return(array[sl_x,sl_y])

In [ ]:
plt.imshow(galaxy_pairs.get_subarray(y_map_array,[1275,500],50),vmax=0.00001,vmin=-0.0001)

In [25]:
top_left_corner = np.array([330,52])
top_right_corner = np.array([330,2468])
bottom_right_corner = np.array([1275,2000])
bottom_left_corner = np.array([1275,2520-2000])

In [26]:
edges = np.vstack((top_left_corner,top_right_corner,bottom_left_corner,bottom_right_corner))

In [27]:
edges

array([[ 330,   52],
       [ 330, 2468],
       [1275,  520],
       [1275, 2000]])

In [28]:
temp1 = sptpol_software.observation.sky.pix2Ang(top_left_corner,np.array([0,-57.5]),reso_arcmin=0.25,map_pixel_shape=np.array([1320, 2520]))
print(temp1[0]%360,temp1[1]%360)
print(temp1)

(350.97295132649253, 303.8729166666667)
(-9.027048673507457, -56.12708333333333)


In [29]:
sptpol_software.observation.sky.ang2Pix((-9.027, -56.127),[0,-57.5],reso_arcmin=0.25,map_pixel_shape=np.array([1320,2520]))

([array([330]), array([52])], (array([ True]), array([ True])))

In [30]:
temp2 = sptpol_software.observation.sky.pix2Ang(top_right_corner,np.array([0,-57.5]),reso_arcmin=0.25,map_pixel_shape=np.array([1320, 2520]))

In [31]:
temp3 = sptpol_software.observation.sky.pix2Ang(bottom_left_corner,np.array([0,-57.5]),reso_arcmin=0.25,map_pixel_shape=np.array([1320, 2520]))

In [32]:
temp4 = sptpol_software.observation.sky.pix2Ang(bottom_right_corner,np.array([0,-57.5]),reso_arcmin=0.25,map_pixel_shape=np.array([1320, 2520]))

In [33]:
edges_ang = (temp1,temp2,temp3,temp4)
print(edges_ang)

((-9.027048673507457, -56.12708333333333), (9.034524490214293, -56.12708333333333), (-6.174558841480863, -60.06458333333333), (6.182908481564001, -60.06458333333333))


In [ ]:
# from ..constants import DTOR, RTOD
# def pixel_2_angle(pixel_coords,ra_dec_centre,reso_arcmin,map_pixel_shape):
    
    
#     pixel_coords = (y_coord, x_coord) = pixel_coords[0].astype(float), pixel_coords[1].astype(float)
#     n_pixels = map_pixel_shape.astype(float)
    
#     y_coord = (y_coord + 0.5 - 0.5 * n_pixels[0]) * reso_arcmin / 60
#     x_coord = (x_coord + 0.5 - 0.5 * n_pixels[1]) * reso_arcmin / 60
    
    

In [ ]:
print(max(df['RA']))
print(min(df['RA']))
print(max(df['DEC']))
print(min(df['DEC']))


In [19]:
new_df = df[(df.DEC < -56) & (df.DEC > -60) & (df.RA > -9) & (df.RA < 9)]
new_df = new_df.reset_index(drop=True)

In [20]:
cut_pairs = galaxy_pairs.getPairs(new_df,20)
cut_pairs_df = pd.DataFrame(cut_pairs.T,columns = ['First_Loc','Second_Loc','Sep'])
cut_pairs_df['First_Loc'] = cut_pairs_df.First_Loc.astype(int)
cut_pairs_df['Second_Loc'] = cut_pairs_df.Second_Loc.astype(int)
cut_pairs_df

,First_Loc,Second_Loc,Sep
0,3532,3535,18.290912
1,3509,3515,15.751454
2,3507,3509,16.423754
3,3485,3492,18.048513
4,3492,3500,13.458324
5,3484,3494,19.154830
6,3479,3484,15.340826
7,3467,3479,19.375618
8,3460,3466,11.133920
9,3457,3460,12.927909
